In this notebook, I build upon the exploration of SnowDepth notebook to estimate snowdepth over timeseries of files available at MCS. I use assumed incidence angle

In [19]:
##import packages
import asf_search as asf
from uavsar_pytools.UavsarImage import UavsarImage #single image class
from uavsar_pytools.uavsar_tools import grd_tiff_convert #single image class
from uavsar_pytools.UavsarScene import UavsarScene # binary convert to .tiffs
from datetime import datetime
import progressbar
from tqdm import tqdm
import logging
import glob
import requests
import zipfile

#Files/directory package
import os

#plotting packages
import matplotlib.pyplot as plt
import seaborn as sns

#spatial data and other packages
import pandas as pd
import geopandas as gpd
import rioxarray 
#import earthpy as et
import numpy as np
#import rasterio
#from rasterio.warp import calculate_default_transform, reproject, Resampling
import glob

In [11]:
#set path
print('working directory:', os.getcwd()) #print current working directory

#set the absolute path for the data
data_path = '/SNOWDATA/Nah/lowman/tiffs/'
print('data directory:', data_path)

working directory: /home/naheemadebisi/snow-analytics/lowman
data directory: /SNOWDATA/Nah/lowman/tiffs/


In [12]:
# Confirm the file for each time stamps (compare with name at UAF)
lowman_jan15_jan20 = data_path + 'lowman_23205_21002-004_21004-003_0005d_s01_L090VV_01.unw.grd.tiff' #UA_lowman_23205_21002-004_21004-003_0005d_s01_L090_01
lowman_jan20_jan27 = data_path + 'lowman_23205_21004-003_21008-000_0007d_s01_L090VV_01.unw.grd.tiff' #UA_lowman_23205_21004-003_21008-000_0007d_s01_L090_01
lowman_jan27_feb03 = data_path + 'lowman_23205_21008-000_21009-004_0007d_s01_L090VV_01.unw.grd.tiff' #UA_lowman_23205_21008-000_21009-004_0007d_s01_L090_01
lowman_feb03_feb10 = data_path + 'lowman_23205_21009-004_21012-000_0007d_s01_L090VV_01.unw.grd.tiff' #UA_lowman_23205_21009-004_21012-000_0007d_s01_L090_01
#lowman_feb03_feb10_b = data_path + 'lowman_05208_21009-005_21012-004_0007d_s01_L090VV_01.unw.grd.tiff' #UA_lowman_05208_21009-005_21012-004_0007d_s01_L090_01
#lowman_feb10_mar03 = data_path + 'lowman_23205_21012-000_21015-008_0021d_s01_L090VV_01.unw.grd.tiff' #No unw file #UA_lowman_23205_21012-000_21015-008_0021d_s01_L090_01
#lowman_feb10_mar03_b = data_path + 'lowman_05208_21012-004_21015-009_0021d_s01_L090VV_01.unw.grd.tiff' #UA_lowman_05208_21012-004_21015-009_0021d_s01_L090_01
lowman_mar03_mar10 = data_path + 'lowman_23205_21015-008_21017-018_0007d_s01_L090VV_01.unw.grd.tiff' #UA_lowman_23205_21015-008_21017-018_0007d_s01_L090_01
#lowman_mar03_mar10_b = data_path + 'lowman_05208_21015-009_21017-019_0007d_s01_L090VV_01.unw.grd.tiff' #UA_lowman_05208_21015-009_21017-019_0007d_s01_L090_01
lowman_mar10_mar16 = data_path + 'lowman_23205_21017-018_21019-018_0006d_s01_L090VV_01.unw.grd.tiff' #UA_lowman_23205_21017-018_21019-018_0006d_s01_L090_01 
#lowman_mar10_mar16_b = data_path + 'lowman_05208_21017-019_21019-019_0006d_s01_L090VV_01.unw.grd.tiff' #UA_lowman_05208_21017-019_21019-019_0006d_s01_L090_01
lowman_mar16_mar22 = data_path + 'lowman_23205_21019-018_21021-006_0006d_s01_L090VV_01.unw.grd.tiff' #UA_lowman_23205_21019-018_21021-006_0006d_s01_L090_01
#lowman_mar16_mar22_b = data_path + 'lowman_05208_21019-019_21021-007_0006d_s01_L090VV_01.unw.grd.tiff' #UA_lowman_05208_21019-019_21021-007_0006d_s01_L090_01


unw_files = [lowman_jan15_jan20, lowman_jan20_jan27, lowman_jan27_feb03, lowman_feb03_feb10, 
             lowman_mar03_mar10, lowman_mar10_mar16, lowman_mar16_mar22]

#Put as list of string for acquisition period attribute
acquisition_period = ['Jan15-Jan20', 'Jan20-Jan27', 'Jan27-Feb03', 'Feb03-Feb10', 'Mar03-Mar10', 'Mar10-Mar16', 'Mar16-Mar22']
acquisition_period_b = ['01-15-2021_01-20-2021', '01-20-2021_01-27-2021', '01-27-2021_02-03-2021', '02-03-2021_02-10-2021',  
                        '03-03-2021_03-10-2021', '03-10-2021_03-16-2021', '03-16-2021_03-22-2021']



In [13]:
#Read each file into as x array DataArray object, give a better name and add the metadata of interferogram date period
unw_files_arr = []
#unw_files_arr_whisker = []
snow_depth_arr = []
#roi_snow_depth = []
for i in range(len(unw_files)):
    unw_files_arr.append(rioxarray.open_rasterio(unw_files[i]))
    unw_files_arr[i].name = acquisition_period[i]
    unw_files_arr[i].attrs['acquisition_period'] = acquisition_period[i]
    unw_files_arr[i].attrs['acquisition_period_b'] = acquisition_period_b[i]
    unw_files_arr[i].attrs['sensor_polarization'] = 'VV'
    unw_files_arr[i].attrs['location'] = 'lowman'

    # Remove Outlier: Select only data values within 0.95 and 0.05 of the upper whisker
    #unw_arr_whisker = unw_files_arr[i].where(((unw_files_arr[i] < float(unw_files_arr[i].quantile(0.95).values)) & (unw_files_arr[i] > float(unw_files_arr[i].quantile(0.05).values))), other = np.nan)
    #unw_files_arr_whisker.append(unw_arr_whisker)
    
    #calculate the snow depth based on Guinerrisen 2001 formula
    snowdepth_cm = (-1 * unw_files_arr[i] * 24.0)/(4 * np.pi * (np.cos(0.25)  - np.sqrt(1.5 - (np.sin(0.25)**2))))
    snow_depth_arr.append(snowdepth_cm)

    # # Append snow depth at the point of interest. MCS Snotel site [lat, lon] = [43.9333, -115.667]
    # roi_snow_depth.append((snowdepth_cm.sel(x = -115.667, y = 43.9333, method = 'nearest')).to_dataframe())

In [17]:
# Snow depth at the point of interest. MCS Snotel site [lat, lon] = [43.9333, -115.667]
snow_depth_arr[0].sel(x = -115.667, y = 43.9333, method = 'nearest')

<xarray.DataArray 'Jan15-Jan20' (band: 1)>
array([nan], dtype=float32)
Coordinates:
  * band         (band) int64 1
    x            float64 -115.7
    y            float64 43.93
    spatial_ref  int64 0

In [16]:
# save xarray to tif file
snow_depth_arr[0].rio.to_raster('/home/naheemadebisi/snow-analytics/lowman/MCS/MCS_15_20Jan2021_snow_diff.tif')
snow_depth_arr[1].rio.to_raster('/home/naheemadebisi/snow-analytics/lowman/MCS/MCS_20_27Jan2021_snow_diff.tif')
snow_depth_arr[2].rio.to_raster('/home/naheemadebisi/snow-analytics/lowman/MCS/MCS_27_03Feb2021_snow_diff.tif')
snow_depth_arr[3].rio.to_raster('/home/naheemadebisi/snow-analytics/lowman/MCS/MCS_03_10Feb2021_snow_diff.tif')
snow_depth_arr[4].rio.to_raster('/home/naheemadebisi/snow-analytics/lowman/MCS/MCS_03_10Mar2021_snow_diff.tif')
snow_depth_arr[5].rio.to_raster('/home/naheemadebisi/snow-analytics/lowman/MCS/MCS_10_16Mar2021_snow_diff.tif')
snow_depth_arr[6].rio.to_raster('/home/naheemadebisi/snow-analytics/lowman/MCS/MCS_16_22Mar2021_snow_diff.tif')
